# Customer.IO People Management

This notebook demonstrates how to manage people (users) using the Customer.IO API client library. It covers user identification, profile updates, suppression/unsuppression, and deletion operations.

## Databricks Environment SetupThis section handles the installation of required libraries and configuration for Databricks clusters. Skip this section if running locally.

In [ ]:
# Databricks library installation# Uncomment the following lines when running on Databricks# %pip install customerio-api-client# OR if using a wheel file:# %pip install /dbfs/path/to/customerio_api_client-1.0.0-py3-none-any.whl# For development without packaging, install individual requirements:# %pip install httpx>=0.25.0 pydantic>=2.0.0 pandas>=2.0.0 python-dotenv>=1.0.0 structlog>=24.0.0# Restart kernel after installation if needed# import dbutils# dbutils.library.restartPython()

## Setup and Imports

Import the people management functions and initialize the API client.

In [ ]:
import osimport sysfrom datetime import datetime# For local development with utils imports (backwards compatibility)# Add src directory to path if it existsnotebook_dir = os.path.dirname(os.path.abspath(''))project_root = os.path.abspath(os.path.join(notebook_dir, '..', '..'))src_path = os.path.join(project_root, 'src')if os.path.exists(src_path):    sys.path.insert(0, src_path)    # Try importing from utils (local development pattern)    try:        from utils.api_client import CustomerIOClient        from utils.people_manager import (            identify_user,            delete_user,            suppress_user,            unsuppress_user        )        from utils.exceptions import CustomerIOError, ValidationError    except ImportError:        # Fall back to package imports        from pipelines_api.api_client import CustomerIOClient        from pipelines_api.people_manager import (            identify_user,            delete_user,            suppress_user,            unsuppress_user        )        from pipelines_api.exceptions import CustomerIOError, ValidationErrorelse:    # Databricks or package installation - use package imports    from pipelines_api.api_client import CustomerIOClient    from pipelines_api.people_manager import (        identify_user,        delete_user,        suppress_user,        unsuppress_user    )    from pipelines_api.exceptions import CustomerIOError, ValidationErrorprint("People management functions imported successfully")

## Client Initialization

Initialize the Customer.IO client with your credentials.

In [ ]:
# Initialize client (use your actual credentials)client = CustomerIOClient(    api_key = dbutils.secrets.get(scope="customer-io", key="api_key")    region = dbutils.secrets.get(scope="customer-io", key="region")    print("Using Databricks secrets for credentials")except:    # Fall back to environment variables    API_KEY = os.getenv('CUSTOMERIO_API_KEY', 'your-api-key-here')    REGION = os.getenv('CUSTOMERIO_REGION', 'us')  # 'us' or 'eu'    print("Using environment variables for credentials"))print(f"Client initialized for region: {client.region}")print(f"Base URL: {client.base_url}")

## User Identification

Identify users and create or update their profiles in Customer.IO.

In [ ]:
# Example user data
user_id = f"demo_user_{int(datetime.now().timestamp())}"

user_traits = {
    "email": "demo@example.com",
    "name": "Demo User",
    "created_at": datetime.now().isoformat(),
    "plan": "premium",
    "company": "Demo Company",
    "location": "San Francisco, CA"
}

print(f"Identifying user: {user_id}")
print(f"User traits: {user_traits}")

In [ ]:
# Identify the user
try:
    response = identify_user(client, user_id, user_traits)
    print("SUCCESS: User identified successfully!")
    print(f"Response: {response}")
except CustomerIOError as e:
    print(f"ERROR: API Error: {e}")
except ValidationError as e:
    print(f"ERROR: Validation Error: {e}")
except Exception as e:
    print(f"ERROR: Unexpected error: {e}")

## Profile Updates

Update user profiles with new information.

In [ ]:
# Update user profile with new traits
updated_traits = {
    "plan": "enterprise",  # Upgraded plan
    "last_login": datetime.now().isoformat(),
    "feature_flags": {
        "beta_features": True,
        "advanced_analytics": True
    }
}

try:
    response = identify_user(client, user_id, updated_traits)
    print("SUCCESS: User profile updated successfully!")
    print(f"Updated traits: {updated_traits}")
except Exception as e:
    print(f"ERROR: Error updating profile: {e}")

## User Suppression and Unsuppression

Suppress users to stop sending them messages, or unsuppress to resume messaging.

In [ ]:
# Suppress a user (stop sending messages)
try:
    response = suppress_user(client, user_id)
    print("SUCCESS: User suppressed successfully!")
    print(f"User {user_id} will no longer receive messages")
    print(f"Response: {response}")
except Exception as e:
    print(f"ERROR: Error suppressing user: {e}")

In [ ]:
# Unsuppress a user (resume sending messages)
try:
    response = unsuppress_user(client, user_id)
    print("SUCCESS: User unsuppressed successfully!")
    print(f"User {user_id} will resume receiving messages")
    print(f"Response: {response}")
except Exception as e:
    print(f"ERROR: Error unsuppressing user: {e}")

## Bulk Operations

Handle multiple users efficiently.

In [ ]:
# Create multiple test users
test_users = []
base_timestamp = int(datetime.now().timestamp())

for i in range(3):
    user_data = {
        "user_id": f"bulk_user_{base_timestamp}_{i}",
        "traits": {
            "email": f"user{i}@example.com",
            "name": f"Bulk User {i}",
            "user_type": "test_user",
            "created_at": datetime.now().isoformat()
        }
    }
    test_users.append(user_data)

print(f"Created {len(test_users)} test users for bulk operations")

In [ ]:
# Identify multiple users
successful_identifications = 0
failed_identifications = 0

for user_data in test_users:
    try:
        response = identify_user(
            client, 
            user_data["user_id"], 
            user_data["traits"]
        )
        successful_identifications += 1
        print(f"SUCCESS: Identified: {user_data['user_id']}")
    except Exception as e:
        failed_identifications += 1
        print(f"ERROR: Failed to identify {user_data['user_id']}: {e}")

print(f"\nBulk identification results:")
print(f"Successful: {successful_identifications}")
print(f"Failed: {failed_identifications}")

## User Deletion

Permanently delete users from Customer.IO. Use with caution as this operation cannot be undone.

In [ ]:
# Delete test users (cleanup)
deleted_users = 0
deletion_errors = 0

# Delete the demo user and bulk test users
users_to_delete = [user_id] + [user["user_id"] for user in test_users]

for user_to_delete in users_to_delete:
    try:
        response = delete_user(client, user_to_delete)
        deleted_users += 1
        print(f"SUCCESS: Deleted: {user_to_delete}")
    except Exception as e:
        deletion_errors += 1
        print(f"ERROR: Failed to delete {user_to_delete}: {e}")

print(f"\nDeletion results:")
print(f"Successfully deleted: {deleted_users}")
print(f"Deletion errors: {deletion_errors}")

## Advanced Examples

More complex user management scenarios.

In [ ]:
# Example: User lifecycle management
lifecycle_user_id = f"lifecycle_user_{int(datetime.now().timestamp())}"

# 1. Create user
initial_traits = {
    "email": "lifecycle@example.com",
    "name": "Lifecycle User",
    "status": "trial",
    "created_at": datetime.now().isoformat()
}

identify_user(client, lifecycle_user_id, initial_traits)
print(f"1. Created user: {lifecycle_user_id}")

# 2. Upgrade to paid
upgrade_traits = {
    "status": "paid",
    "plan": "professional",
    "upgraded_at": datetime.now().isoformat()
}

identify_user(client, lifecycle_user_id, upgrade_traits)
print(f"2. Upgraded user to paid plan")

# 3. Temporarily suppress (e.g., vacation mode)
suppress_user(client, lifecycle_user_id)
print(f"3. Suppressed user (vacation mode)")

# 4. Reactivate
unsuppress_user(client, lifecycle_user_id)
print(f"4. Unsuppressed user (back from vacation)")

# 5. Final update
final_traits = {
    "last_active": datetime.now().isoformat(),
    "vacation_mode": False
}

identify_user(client, lifecycle_user_id, final_traits)
print(f"5. Updated user activity status")

# Cleanup
delete_user(client, lifecycle_user_id)
print(f"6. Cleaned up test user")

## Best Practices

Key recommendations for people management:

**User Identification:**
- Use consistent user IDs across your application
- Include essential traits like email, name, and creation date
- Update profiles incrementally rather than resending all data

**Suppression Management:**
- Suppress users who explicitly opt out of communications
- Use suppression for temporary pauses (vacation, preferences)
- Unsuppress users when they re-engage or update preferences

**Data Privacy:**
- Delete users upon GDPR/CCPA deletion requests
- Be cautious with deletion as it's irreversible
- Consider suppression as an alternative to deletion when appropriate

**Error Handling:**
- Always wrap API calls in try-catch blocks
- Log failures for monitoring and debugging
- Implement retry logic for transient failures

**Performance:**
- Use batch operations for bulk updates (see 05_batch_operations.ipynb)
- Respect rate limits (3000 requests per 3 seconds)
- Consider queueing for high-volume operations

## Next Steps

Explore more Customer.IO functionality:

- **02_event_tracking.ipynb** - Track user events and behaviors
- **03_objects_and_relationships.ipynb** - Manage objects and relationships
- **04_device_management.ipynb** - Handle device registration and updates
- **05_batch_operations.ipynb** - Efficient bulk processing